In [20]:
import pandas as pd
import re

#df : 스크래핑한 호주 문서
df = pd.read_csv("호주_세관_최근10.csv")

df_title = list(df['title'])
df_text = list(df['text'])
df_add = []

for i in range(len(df)):
    df_add.append(df_title[i]+""+df_text[i])

print(len(df_add))

#df_keyword : 선정한 호주 키워드 101개
df_keyword = pd.read_csv("호주_번역_100.csv", index_col = False)
print(len(df_keyword))

10
102


In [21]:
import numpy as np
import itertools
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

# SentenceTransformer : 모델 훈련 위한 라이브러리
# 'distiluse-base-multilingual-cased-v1' : 사용할 모델
model = SentenceTransformer('distiluse-base-multilingual-cased-v1')

In [22]:
#keyword : 불러온 호주 키워드 중 번역 Column 추출
keyword = list(df_keyword["번역"])
print(len(keyword))

102


In [23]:
#키워드 벡터화
keyword_embedding = []

for ele in keyword:
    keyword_embedding.append(model.encode(ele))

print(len(keyword_embedding))

102


In [24]:
#호주 문서 10개 embedding
import nltk
import string
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords 
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

doc_embedding = []
doc_words = []
cnt = 0

for doc in df_add:
    result = []
    sentence = ''
    stop_words = set(stopwords.words('english'))

    doc = doc.replace("\n", "").strip()


    word_tokens = pos_tag(word_tokenize(doc))

    word_tokens = [t[0] for t in word_tokens if t[1] == "NN" and len(t[0]) > 2]
    for w in word_tokens:
        if w not in stop_words:
            w = lemmatizer.lemmatize(w)
            result.append(w)
    result = [word.lower() for word in result]
    doc_words.append(result)
    doc_embedding.append(model.encode(result))
    
    #카운트 추가
    print(cnt, "번 문서 진행 중")
    cnt += 1 

0 번 문서 진행 중
1 번 문서 진행 중
2 번 문서 진행 중
3 번 문서 진행 중
4 번 문서 진행 중
5 번 문서 진행 중
6 번 문서 진행 중
7 번 문서 진행 중
8 번 문서 진행 중
9 번 문서 진행 중


In [25]:
#키워드와 문서 유사도 비교 0.9
doc_keywords_9 = []
for index, doc in enumerate(doc_embedding):
    final_result = []
    for i in range(len(doc)):
        for j in range(len(keyword_embedding)):
            distances = cosine_similarity([doc[i]], [keyword_embedding[j]])
            if distances[0][0]>0.9:
                final_result.append(doc_words[index][i])
# distances = cosine_similarity([doc_embedding[0]], candidate_embeddings)
    print(set(final_result))
    doc_keywords_9.append(set(final_result))

set()
{'car'}
set()
set()
set()
set()
set()
set()
{'plastic'}
set()


In [26]:
#키워드와 문서 유사도 비교 0.8
doc_keywords_8 = []
for index, doc in enumerate(doc_embedding):
    final_result = []
    for i in range(len(doc)):
        for j in range(len(keyword_embedding)):
            distances = cosine_similarity([doc[i]], [keyword_embedding[j]])
            if distances[0][0]>0.8:
                final_result.append(doc_words[index][i])
# distances = cosine_similarity([doc_embedding[0]], candidate_embeddings)
    print(set(final_result))
    doc_keywords_8.append(set(final_result))

set()
{'car', 'measure'}
set()
set()
set()
set()
set()
set()
{'plastic', 'measure'}
set()


In [59]:
#키워드와 문서 유사도 비교 0.75
doc_keywords_75 = []
for index, doc in enumerate(doc_embedding):
    final_result = []
    for i in range(len(doc)):
        for j in range(len(keyword_embedding)):
            distances = cosine_similarity([doc[i]], [keyword_embedding[j]])
            if distances[0][0]>0.75:
                final_result.append(doc_words[index][i])
# distances = cosine_similarity([doc_embedding[0]], candidate_embeddings)
    print(set(final_result))
    doc_keywords_75.append(set(final_result))

{'fuel'}
{'car', 'fuel', 'measure'}
set()
set()
set()
set()
set()
set()
{'fuel', 'plastic', 'measure'}
set()


In [56]:
df_check = pd.DataFrame()
df_check['cosine 0.9'] = doc_keywords_9
df_check['cosine 0.8'] = doc_keywords_8
df_check['cosine 0.75'] = doc_keywords_75

df_check

,cosine 0.9,cosine 0.8,cosine 0.75
0,{},{},"{fuel, duty, refer}"
1,{car},"{car, measure}","{fuel, consumption, measure, vehiclesaustralia..."
2,{},{},{substance}
3,{},{},"{consumption, drop, product, duty, accumulatio..."
4,{},{},{substance}
5,{},{},"{crew, slot, behalf, charterer, cargo, passenger}"
6,{},{},"{industry, destroy, consumption, depot}"
7,{},{},"{treatment, spread, droplet, apply}"
8,{plastic},"{plastic, measure}","{intent, fuel, consignment, measure, resin, pl..."
9,{},{},{substance}
